In [34]:
import os
import sys
import json
import mlflow
from pathlib import Path
from dataclasses import dataclass
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PROJECT_FOLDER"))
from src.utils.common import logger

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    model_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [13]:
os.chdir("..")

In [35]:
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = os.getenv("CONFIG_FILE_PATH"),
        params_filepath: str = os.getenv("PARAMS_FILE_PATH"),
        schema_filepath: str = os.getenv("SCHEMA_FILE_PATH"),
    ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self):
        """
        Get configuration for model training
        """
        config = self.config.model_evaluation
        params = self.params.MultinomialNB
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            test_data_path = Path(config.test_data_path),
            model_path = Path(config.model_path),
            model_params = params,
            metric_file_name = Path(config.metric_file_name),
            target_column = schema.name,
            mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
        )
        
        return model_evaluation_config

In [36]:
import os
import joblib
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [51]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_metrics(self, y_test, y_pred):
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        return precision, recall, f1

    def evaluate_model(self):
        test = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test.drop(columns=[self.config.target_column], axis=1).to_numpy().reshape(-1)
        y_test = test[[self.config.target_column]].to_numpy().reshape(-1)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            y_pred = model.predict(X_test)
            precision, recall, f1 = self.evaluate_metrics(y_test, y_pred)
            scores = {"precision": precision, "recall": recall, "f1_score": f1}
            with open(self.config.metric_file_name, "w") as f:
                json.dump(scores, f)
            mlflow.log_params(self.config.model_params)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, "model", registered_model_name="CountVectorizer+MultinomialNB")
            else:
                mlflow.sklearn.log_model(model, "model")



In [52]:
try:
    configuration_manager = ConfigurationManager()
    model_evaluation = ModelEvaluation(configuration_manager.get_model_evaluation_config())
    model_evaluation.evaluate_model()
except Exception as e:
    logger.error(e)

2024-03-08 17:00:30,115 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\config\config.yaml loaded successfully
2024-03-08 17:00:30,118 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\params.yaml loaded successfully
2024-03-08 17:00:30,121 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\schema.yaml loaded successfully
2024-03-08 17:00:30,122 - sentiment-classifier-logger - INFO - Created directory at: artifacts
2024-03-08 17:00:30,125 - sentiment-classifier-logger - INFO - Created directory at: artifacts/model_evaluation


Successfully registered model 'CountVectorizer+MultinomialNB'.
2024/03/08 17:01:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CountVectorizer+MultinomialNB, version 1
Created version '1' of model 'CountVectorizer+MultinomialNB'.
